In [1]:
"""part2_diffusion_p2.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1... # Placeholder, adjust if needed

# Acknowledgement

Parts of this pset were inspired by
* Berkeley CS294-158, taught by Pieter Abbeel, Wilson Yan, Kevin Frans, and Philipp Wu;
* MIT 6.S184/6.S975, taught by Peter Holderrieth and Ezra Erives;
* The [blog post](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/) about diffusion models by Lilian Weng.


# Submission Guideline for Part 2

Please include your answer to all problems, including formulas, proofs, and the figures generated in each problem, excluding code. You are required to submit the (single) pdf and all (four) notebooks (one for each problem) with your code and running outputs. Do not include code in the pdf file.

Specifically, for Problem 2 in this notebook, the pdf should contain:
- The generated figures `results/p2_train_plot.png` and `results/p2_toy_samples.png`

# Problem 2: Training Diffusion Models on a Toy Dataset
In this problem, we will write the code for training and sampling from a diffusion model on a 2D toy dataset. This part requires GPUs--you can use Google Colab for GPU access. To work on this notebook in Google Colab, copy the `pset-5` directory to your Google Drive and open this notebook. Then, start working on a GPU machine with `Runtime -> Change runtime type -> T4 GPU`.

## Data Generation and Visualization
"""

'part2_diffusion_p2.ipynb\n\nAutomatically generated by Colaboratory.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1... # Placeholder, adjust if needed\n\n# Acknowledgement\n\nParts of this pset were inspired by\n* Berkeley CS294-158, taught by Pieter Abbeel, Wilson Yan, Kevin Frans, and Philipp Wu;\n* MIT 6.S184/6.S975, taught by Peter Holderrieth and Ezra Erives;\n* The [blog post](https://lilianweng.github.io/posts/2021-07-11-diffusion-models/) about diffusion models by Lilian Weng.\n\n\n# Submission Guideline for Part 2\n\nPlease include your answer to all problems, including formulas, proofs, and the figures generated in each problem, excluding code. You are required to submit the (single) pdf and all (four) notebooks (one for each problem) with your code and running outputs. Do not include code in the pdf file.\n\nSpecifically, for Problem 2 in this notebook, the pdf should contain:\n- The generated figures `results/p2_train_plot.png` and `results/p

In [3]:
from sklearn.datasets import make_s_curve
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import math
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import os
from os.path import exists, dirname

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cpu


In [5]:
def toy_2d_data(n=100000):
    x, _ = make_s_curve(n, noise=0.1)
    x = x[:, [0, 2]]
    return x.astype('float32')

In [6]:
def visualize_toy_2d_dataset():
    data = toy_2d_data()
    plt.figure()
    plt.scatter(data[:, 0], data[:, 1])
    plt.show()

visualize_toy_2d_dataset() # Commented out for script execution

In [7]:
"""
## Training and Sampling of Diffusion Models

For code simplicity, we will train a continuous-time variant of the diffusion prompt. In practice training objectives and code between discrete-time and continuous-time diffusion models are similar.

Given a data element $x$ and neural net $f_\theta(x, t)$, implement the following diffusion training steps:

0. Construct a class `Diffusion`
1. Sample the diffusion timestep: $t \sim \text{Uniform}(0, 1)$
2. Compute the noise-strength following a cosine schedule: $\alpha_t = \cos\left(\frac{\pi}{2}t\right), \sigma_t = \sin\left(\frac{\pi}{2}t\right)$
3. Sample noise $\epsilon \sim N(0,I)$ (same shape as $x$) and cmpute noised $x_t = \alpha_t x + \sigma_t \epsilon$
4. Estimate $\hat{\epsilon} = f_\theta(x_t, t)$
5. Optimize the loss $L = \lVert \epsilon - \hat{\epsilon} \rVert_2^2$. Here, it suffices to just take the mean over all dimensions.

Note that for the case of continuous-time diffusion, the forward process is $x_{0\to1}$ and reverse process is $x_{1\to0}$

Use an MLP for $f_\theta$ to optimize the loss. You may find the following details helpful.
* Normalize the data using mean and std computed from the train dataset
* Train 100 epochs, batch size 1024, Adam with LR 1e-3 (100 warmup steps, cosine decay to 0)
* MLP with 4 hidden layers and hidden size 64
* Condition on t by concatenating it with input x (i.e. 2D x + 1D t = 3D cat(x, t))
* Training 100 epochs takes about 2 minutes on the Google Colab T4 GPU.


To sample, implement the standard DDPM sampler. You may find the equation from the [DDIM paper](https://arxiv.org/pdf/2010.02502.pdf) helpful, rewritten and re-formatted here for convenience.
$$x_{t-1} = \alpha_{t-1}\left(\frac{x_t - \sigma_t\hat{\epsilon}}{\alpha_t}\right) + \sqrt{\sigma_{t-1}^2 - \eta_t^2}\hat{\epsilon} + \eta_t\epsilon_t$$
where $\epsilon_t \sim N(0, I)$ is random Gaussian noise. For DDPM, let
$$\eta_t = \sigma_{t-1}/\sigma_t\sqrt{1 - \alpha_t^2/\alpha_{t-1}^2}$$
*Note*: As a sanity check, when $\eta_t$ follows the DDPM setting as shown above, the resulted coefficient of $x_t$ and $\hat{\epsilon}$ should be the same as $A'$ and $B'$ you derived in Problem 1.2 (6) where $\alpha_t$ here corresponds to $\bar{\alpha_t}$ in Problem 1.

To run the reverse process, start from $x_1 \sim N(0, I)$. Then perform `num_steps` DDPM updates (a hyperparameter), pseudocode below.
```
ts = linspace(1 - 1e-4, 1e-4, num_steps + 1)
x = sample_normal
for i in range(num_steps):
    t = ts[i]
    tm1 = ts[i + 1]
    eps_hat = model(x, t)
    x = DDPM_UPDATE(x, eps_hat, t, tm1)
return x
```
*Note*:
* If you encounter NaNs, you may need to clip $\sigma_{t-1}^2 - \eta_t^2$ to 0 if it goes negative, as machine precision issues can make it a very small negative number (e.g. -1e-12) if its too close to 0
* For debugging, you can start with trying small number of epochs. To debug your training code, you can check whether the training and testing losses decrease properly. To debug your sampling code, you can check whether the generated distribution is close to the S-shape target distribution with large `num_steps`.
* To check your answer, the final text loss is roughly around 0.4.
"""

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/var/folders/qx/4sngfkkj5kv2y_sh8nnhj1yr0000gn/T/ipykernel_12721/2168152498.py:1: SyntaxWarning: invalid escape sequence '\s'
  """


"\n## Training and Sampling of Diffusion Models\n\nFor code simplicity, we will train a continuous-time variant of the diffusion prompt. In practice training objectives and code between discrete-time and continuous-time diffusion models are similar.\n\nGiven a data element $x$ and neural net $f_\theta(x, t)$, implement the following diffusion training steps:\n\n0. Construct a class `Diffusion`\n1. Sample the diffusion timestep: $t \\sim \text{Uniform}(0, 1)$\n2. Compute the noise-strength following a cosine schedule: $\x07lpha_t = \\cos\\left(\x0crac{\\pi}{2}t\right), \\sigma_t = \\sin\\left(\x0crac{\\pi}{2}t\right)$\n3. Sample noise $\\epsilon \\sim N(0,I)$ (same shape as $x$) and cmpute noised $x_t = \x07lpha_t x + \\sigma_t \\epsilon$\n4. Estimate $\\hat{\\epsilon} = f_\theta(x_t, t)$\n5. Optimize the loss $L = \\lVert \\epsilon - \\hat{\\epsilon} \rVert_2^2$. Here, it suffices to just take the mean over all dimensions.\n\nNote that for the case of continuous-time diffusion, the for

--- Diffusion Model Components ---

In [8]:
def alpha_sigma_schedule(t):
    """Cosine schedule for alpha_t and sigma_t."""
    alpha_t = torch.cos(math.pi / 2 * t)
    sigma_t = torch.sin(math.pi / 2 * t)
    return alpha_t, sigma_t

In [9]:
class Diffusion:
    def __init__(self, model, data_shape):
        """
        model: neural network to estimate eps_hat (MLP in this problem)
        data_shape: size of the input data, (2,) in this case
        """
        self.model = model.to(DEVICE)
        self.data_shape = data_shape

    def loss(self, x):
        """
        x: the input data (without adding noise) from the dataloader
        Return:
            The loss (as a scalar averaged over all data in the batch)
        """
        # 1. Sample t ~ Uniform(0, 1)
        t = torch.rand(x.shape[0], device=DEVICE) # Sample one t per data point in the batch

        # 2. Compute alpha_t, sigma_t
        alpha_t, sigma_t = alpha_sigma_schedule(t)
        alpha_t = alpha_t.view(-1, *([1] * (x.dim() - 1))) # Reshape for broadcasting
        sigma_t = sigma_t.view(-1, *([1] * (x.dim() - 1)))

        # 3. Sample epsilon ~ N(0, I) and compute x_t
        epsilon = torch.randn_like(x, device=DEVICE)
        x_t = alpha_t * x + sigma_t * epsilon

        # 4. Estimate eps_hat
        eps_hat = self.model(x_t, t) # Pass t directly

        # 5. Compute loss L = ||epsilon - eps_hat||^2 (mean over batch and dims)
        loss = torch.mean((epsilon - eps_hat)**2)
        return loss

    @torch.no_grad()
    def sample(self, n, num_steps):
        """
        n: number of samples to generate
        num_steps: number of steps in the diffusion sampling
        Return:
            The generated sample. Tensor with shape (n, *self.data_shape)
        """
        # Define timesteps
        ts = torch.linspace(1.0 - 1e-4, 1e-4, num_steps + 1, device=DEVICE)

        # Start with x1 ~ N(0, I)
        x = torch.randn(n, *self.data_shape, device=DEVICE)

        self.model.eval() # Ensure model is in eval mode

        # DDPM sampling loop
        for i in range(num_steps):
            t_ = ts[i]
            tm1_ = ts[i + 1]

            # Prepare t and tm1 tensors for the model and schedule
            t = torch.full((n,), t_, device=DEVICE)
            tm1 = torch.full((n,), tm1_, device=DEVICE)

            # Get alpha and sigma for t and t-1
            alpha_t, sigma_t = alpha_sigma_schedule(t)
            alpha_tm1, sigma_tm1 = alpha_sigma_schedule(tm1)

            # Reshape for broadcasting
            alpha_t = alpha_t.view(-1, *([1] * (x.dim() - 1)))
            sigma_t = sigma_t.view(-1, *([1] * (x.dim() - 1)))
            alpha_tm1 = alpha_tm1.view(-1, *([1] * (x.dim() - 1)))
            sigma_tm1 = sigma_tm1.view(-1, *([1] * (x.dim() - 1)))

            # Predict epsilon_hat
            eps_hat = self.model(x, t_) # Model expects scalar t per batch item? No, pass t tensor

            # Calculate eta_t for DDPM
            # eta_t = sigma_{t-1}/sigma_t * sqrt(1 - alpha_t^2/alpha_{t-1}^2)
            # Avoid division by zero if alpha_tm1 is close to 0 (i.e., tm1 is close to 1)
            ratio_alpha_sq = (alpha_t / alpha_tm1)**2
            # Clip ratio_alpha_sq to prevent issues near t=1 where alpha_tm1 -> 0
            ratio_alpha_sq = torch.clamp(ratio_alpha_sq, max=1.0 - 1e-6) # Ensure sqrt arg is non-negative
            eta_t_sq = (sigma_tm1 / sigma_t)**2 * (1 - ratio_alpha_sq)
            eta_t = torch.sqrt(torch.clamp(eta_t_sq, min=1e-8)) # Clip for sqrt and avoid division by zero if sigma_t is small

            # Calculate term for epsilon_hat coefficient: sqrt(sigma_{t-1}^2 - eta_t^2)
            # sigma_tm1_sq_minus_eta_t_sq = sigma_tm1**2 - eta_t**2 # This can be negative due to precision
            # From derivation, sqrt(sigma_{t-1}^2 - eta_t^2) = sqrt(sigma_{t-1}^2 * (1 - (sigma_{t-1}^2 / sigma_t^2 * (1 - alpha_t^2 / alpha_{t-1}^2)) / sigma_{t-1}^2))
            # simplifies to sigma_{t-1} * sqrt(1 - (sigma_{t-1}^2 / sigma_t^2 * (1 - alpha_t^2 / alpha_{t-1}^2)))
            # Let's rewrite the update step using the formula directly for clarity and check with P1.2
            # x_{t-1} = A' * x_t + B' * eps_hat + C' * noise
            # From P1.2 (continuous equivalent):
            # A' = alpha_{t-1} / alpha_t
            # B' = sigma_{t-1} * sqrt(1 - alpha_t^2 / alpha_{t-1}^2) - alpha_{t-1} * sigma_t / alpha_t # Check this coefficient
            # C' = eta_t

            # Using the DDIM paper formula provided:
            # x_{t-1} = alpha_{t-1} * ( (x_t - sigma_t * eps_hat) / alpha_t ) + sqrt(sigma_{t-1}^2 - eta_t^2) * eps_hat + eta_t * eps_t
            term1 = alpha_tm1 * ((x - sigma_t * eps_hat) / alpha_t)

            # Calculate sigma_{t-1}^2 - eta_t^2 carefully
            # eta_t^2 = sigma_tm1^2 / sigma_t^2 * (1 - alpha_t^2 / alpha_tm1^2)
            sigma_tm1_sq_minus_eta_t_sq = sigma_tm1**2 - (sigma_tm1**2 / sigma_t**2) * (1 - ratio_alpha_sq)
            # Clip to avoid sqrt of negative
            sigma_tm1_sq_minus_eta_t_sq_clipped = torch.clamp(sigma_tm1_sq_minus_eta_t_sq, min=0.0)
            term2_coeff = torch.sqrt(sigma_tm1_sq_minus_eta_t_sq_clipped)
            term2 = term2_coeff * eps_hat

            # Sample random noise eps_t ~ N(0, I) only if eta_t > 0
            term3 = 0.0
            if tm1_ > 1e-4: # Only add noise if not the last step (t-1 is not approx 0)
                eps_t = torch.randn_like(x, device=DEVICE)
                term3 = eta_t * eps_t

            # DDPM Update
            x = term1 + term2 + term3

        self.model.train() # Switch back to train mode
        return x

    def __getattr__(self, name):
        if name in ['train', 'eval', 'parameters', 'state_dict', 'load_state_dict']:
            return getattr(self.model, name)
        return self.__getattribute__(name)

In [10]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_hidden_layers, timestep_dim=1):
        super().__init__()
        self.timestep_dim = timestep_dim
        prev_dim = input_dim + timestep_dim
        net = []
        dims = [hidden_dim] * n_hidden_layers + [input_dim]
        for i, dim in enumerate(dims):
            net.append(nn.Linear(prev_dim, dim))
            if i < len(dims) - 1:
                net.append(nn.ReLU())
            prev_dim = dim
        self.net = nn.Sequential(*net)

    def forward(self, x, t):
        x = torch.cat([x, t.reshape(-1, 1)], dim=1) # Ensure t has shape (batch_size, 1)
        return self.net(x)

--- Training Loop Components ---

In [11]:
def get_lr(step, total_steps, warmup_steps, lr_init, use_cos_decay):
    """
    Function that returns the learning rate for the specific step.
    Handles linear warmup and optional cosine decay.
    """
    if step < warmup_steps:
        # Linear warmup
        lr = lr_init * (step + 1) / warmup_steps
    elif use_cos_decay:
        # Cosine decay after warmup
        progress = (step - warmup_steps) / max(1, total_steps - warmup_steps) # Avoid division by zero
        lr = lr_init * 0.5 * (1.0 + math.cos(math.pi * progress))
    else:
        # Constant learning rate after warmup
        lr = lr_init
    # Ensure lr doesn't go below a minimum value if using cosine decay (e.g., decay to 0)
    if use_cos_decay:
         # The formula naturally decays towards 0 at total_steps
         pass # lr can be 0 at the end
    return lr

In [ ]:
def train(model, train_loader, optimizer, scheduler):
    """
    model: model to train, the Diffusion class in this case.
    train_loader: dataloader for the train_data after normalization
    optimizer: use torch.optim.Adam
    scheduler: use optim.lr_scheduler.LambdaLR
    Return:
        Tensor with train loss of each batch
    """
    model.train()
    train_losses = []
    for batch in tqdm(train_loader, desc="Training Batch", leave=False):
        x = batch[0].to(DEVICE) # Assuming dataloader yields tuples/lists

        optimizer.zero_grad()
        loss = model.loss(x)
        loss.backward()
        optimizer.step()
        scheduler.step() # Step the scheduler each batch

        train_losses.append(loss.item())

    return torch.tensor(train_losses)

In [ ]:
@torch.no_grad()
def eval_loss(model, data_loader):
    """
    model: model to train, the Diffusion class in this case.
    data_loader: dataloader for the test_data after normalization
    Return:
        Scalar with the average test loss of each batch
    """
    model.eval()
    total_loss = 0.0
    count = 0
    for batch in data_loader:
        x = batch[0].to(DEVICE)
        loss = model.loss(x)
        total_loss += loss.item() * x.shape[0]
        count += x.shape[0]
    return total_loss / count

In [ ]:
def train_epochs(model, train_loader, test_loader, train_args):
    """
    model: model to train, the Diffusion class in this case.
    train_loader: dataloader for the train_data after normalization
    test_loader: dataloader for the test_data after normalization
    Return:
        Two np.array for all the train losses and test losses at each step
    """
    epochs, lr_init = train_args['epochs'], train_args['lr']
    warmup_steps = train_args.get('warmup', 0)
    use_cos_decay = train_args.get('use_cos_decay', False)

    optimizer = optim.Adam(model.parameters(), lr=lr_init)

    total_steps = epochs * len(train_loader)

    # Define scheduler using get_lr
    scheduler = optim.lr_scheduler.LambdaLR(
        optimizer,
        lr_lambda=lambda step: get_lr(step, total_steps, warmup_steps, lr_init, use_cos_decay)
    )

    all_train_losses = []
    all_test_losses = []

    # Initial test loss
    initial_test_loss = eval_loss(model, test_loader)
    all_test_losses.append(initial_test_loss)
    print(f"Epoch 0 (Initial), Test Loss: {initial_test_loss:.4f}")

    for epoch in range(epochs):
        epoch_train_losses = train(model, train_loader, optimizer, scheduler)
        all_train_losses.extend(epoch_train_losses.cpu().numpy()) # Store individual batch losses

        epoch_test_loss = eval_loss(model, test_loader)
        all_test_losses.append(epoch_test_loss)

        print(f"Epoch {epoch + 1}/{epochs}, Test Loss: {epoch_test_loss:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

    return np.array(all_train_losses), np.array(all_test_losses)

--- Main Function and Plotting ---

In [ ]:
def toy_diffusion(train_data, test_data):
    """
    train_data: A (100000, 2) numpy array of 2D points
    test_data: A (10000, 2) numpy array of 2D points

    Returns
    - a (# of training iterations,) numpy array of train losses evaluated every minibatch
    - a (# of num_epochs + 1,) numpy array of test losses evaluated at the start of training and the end of every epoch
    - a numpy array of size (9, 2000, 2) of samples drawn from your model.
      Draw 2000 samples for each of 9 different number of diffusion sampling steps
      of evenly logarithmically spaced integers 1 to 256 (corrected from 512 based on hint)
      hint: np.power(2, np.linspace(0, 8, 9)).astype(int)
    - train_data_mean: np.array used for normalization
    - train_data_std: np.array used for normalization
    """
    # --- Hyperparameters ---
    batch_size = 1024
    epochs = 100
    lr = 1e-3
    hidden_dim = 64
    n_hidden_layers = 4
    warmup_steps = 100
    use_cos_decay = True
    num_samples_to_gen = 2000
    sample_steps_list = np.power(2, np.linspace(0, 8, 9)).astype(int) # 1, 2, 4, ..., 256

    # --- Data Preparation ---
    # Normalize data
    train_data_mean = train_data.mean(axis=0)
    train_data_std = train_data.std(axis=0)
    train_data_normalized = (train_data - train_data_mean) / train_data_std
    test_data_normalized = (test_data - train_data_mean) / train_data_std

    # Create TensorDatasets and DataLoaders
    train_dataset = TensorDataset(torch.from_numpy(train_data_normalized))
    test_dataset = TensorDataset(torch.from_numpy(test_data_normalized))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # --- Model Initialization ---
    data_shape = train_data.shape[1:] # Should be (2,)
    input_dim = np.prod(data_shape).item() # Should be 2

    mlp_model = MLP(input_dim=input_dim, hidden_dim=hidden_dim, n_hidden_layers=n_hidden_layers)
    diffusion_model = Diffusion(model=mlp_model, data_shape=data_shape)

    # --- Training ---
    train_args = {
        'epochs': epochs,
        'lr': lr,
        'warmup': warmup_steps,
        'use_cos_decay': use_cos_decay
    }

    print("Starting training...")
    all_train_losses, all_test_losses = train_epochs(
        diffusion_model,
        train_loader,
        test_loader,
        train_args
    )
    print("Training finished.")

    # --- Sampling ---
    print("Generating samples...")
    all_samples = []
    diffusion_model.eval() # Ensure model is in eval mode for sampling
    for num_steps in tqdm(sample_steps_list, desc="Sampling Steps"):
        samples_normalized = diffusion_model.sample(n=num_samples_to_gen, num_steps=num_steps)
        # Samples are on DEVICE, move to CPU and convert to numpy
        all_samples.append(samples_normalized.cpu().numpy())

    all_samples_np = np.stack(all_samples) # Shape: (9, num_samples_to_gen, 2)

    # We need to return the mean and std for denormalization during plotting
    return all_train_losses, all_test_losses, all_samples_np, train_data_mean, train_data_std

In [ ]:
def savefig(fname: str, show_figure: bool = True) -> None:
    if not exists(dirname(fname)):
        os.makedirs(dirname(fname))
    plt.tight_layout()
    plt.savefig(fname)
    if show_figure:
        plt.show()
    plt.close() # Close the figure after saving/showing

In [ ]:
def save_training_plot(
    train_losses: np.ndarray, test_losses: np.ndarray, title: str, fname: str
) -> None:
    plt.figure()
    n_epochs = len(test_losses) - 1
    x_train = np.linspace(0, n_epochs, len(train_losses))
    x_test = np.arange(n_epochs + 1)

    plt.plot(x_train, train_losses, label="train loss")
    plt.plot(x_test, test_losses, label="test loss")
    plt.legend()
    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Loss (MSE)") # Usually MSE for diffusion
    savefig(fname)

In [ ]:
def save_multi_scatter_2d(data: np.ndarray, data_mean=None, data_std=None) -> None:
    fig, axs = plt.subplots(3, 3, figsize=(10, 10))
    # num_steps = np.power(2, np.linspace(0, 9, 9)).astype(int) # This includes 2^9 = 512
    num_steps = np.power(2, np.linspace(0, 8, 9)).astype(int) # Correcting based on hint, goes up to 2^8 = 256

    if data_mean is not None and data_std is not None:
        # Denormalize data for plotting
        data = data * data_std + data_mean

    for i in range(3):
        for j in range(3):
            idx = i * 3 + j
            axs[i, j].scatter(data[idx, :, 0], data[idx, :, 1], s=5, alpha=0.7) # Adjust point size and alpha
            axs[i, j].set_title(f'Steps = {num_steps[idx]}')
            axs[i, j].set_aspect('equal', adjustable='box') # Ensure aspect ratio is equal
            axs[i, j].set_xlim(-2.5, 2.5) # Set consistent limits if possible
            axs[i, j].set_ylim(-1.5, 1.5)
    savefig("results/p2_toy_samples.png")

In [ ]:
def toy_save_results(fn):
    train_data = toy_2d_data(n=100000)
    test_data = toy_2d_data(n=10000)
    train_losses, test_losses, samples, data_mean, data_std = fn(train_data, test_data) # fn needs to return mean/std

    print(f"Final Test Loss: {test_losses[-1]:.4f}")

    save_training_plot(
        train_losses,
        test_losses,
        f"P2 Train Plot",
        f"results/p2_train_plot.png"
    )

    save_multi_scatter_2d(samples, data_mean, data_std) # Pass mean/std for denormalization

In [ ]:
if __name__ == "__main__":
    toy_save_results(toy_diffusion)

In [ ]:
"""
# Submission Guideline for Part 2

Please include your answer to all problems, including formulas, proofs, and the figures generated in each problem, excluding code. You are required to submit the (single) pdf and all (four) notebooks (one for each problem) with your code and running outputs. Do not include code in the pdf file.

Specifically, for Problem 2 in this notebook, the pdf should contain:
- The generated figures `results/p2_train_plot.png` and `results/p2_toy_samples.png`
"""